<a href="https://www.kaggle.com/code/hassangul/twitter-sentiment-analysis?scriptVersionId=117466087" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Liberaries will be used**

In [1]:
import pandas as pd


#Libraries for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")

# Data Understanding and Analysis

In [2]:
# Data set for validation
val=pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv", header=None)

#Full dataset for Train-Test
train=pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv", header=None)

In [3]:
train.head(5)

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       74682 non-null  int64 
 1   1       74682 non-null  object
 2   2       74682 non-null  object
 3   3       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [5]:
# 1 refers to colum one in the data set which contain entities (those bodies/entities/companies twittes are about) [checking unique entities]
len(train[1].unique())

32

In [6]:
# column 2 are labels/target [checking how many different targest/labels are there]
train[2].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [7]:
# Giving columns understandible names
train.rename(columns={2:"labels",1:"entity",3:"tweets"}, inplace=True)

In [8]:
# replacing long name to simple (PlayerUnknownsBattlegrounds(PUBG) to PUBG) for easynes 
train["entity"].replace(to_replace="PlayerUnknownsBattlegrounds(PUBG)", value="PUBG", inplace=True)

In [9]:
train.head()

,0,entity,labels,tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [10]:
# checking frequency of the different targets/labels in data set
targets = np.array(train["labels"])

frequency_dict = {}

for i in targets:
    if i not in frequency_dict:
        frequency_dict[i] = 1
    else:
        frequency_dict[i] +=1

# storing keys(tragets/labels) sepratly form there frequencies
labels = list(frequency_dict.keys())
frequency = list(frequency_dict.values())

In [11]:
# creating graph to see the frequency of targets visualy 

fig=px.bar(x=labels,y=frequency)
fig.update_layout(title={
        'text': "Frequency of target",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},xaxis_title='Targets',yaxis_title='Frequency')

In [12]:
# counting frequencies of the entities 
targets = np.array(train["entity"])

frequency_dict = {}

for i in targets:
    if i not in frequency_dict:
        frequency_dict[i] = 1
    else:
        frequency_dict[i] +=1
    
labels = []
frequency = list(frequency_dict.values())

for i in frequency_dict.keys():
    labels.append(i.strip())

# just ploting the above calculated frequencies of the entities
fig=px.bar(x=labels,y=frequency)

fig.update_layout(title={
        'text': "Frequency of entities",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},xaxis_title='Entities',yaxis_title='Frequency')

In [13]:
"""created class so, i can count frequencies of different target/labels according to entites using same code instead of writing 
code for positve, negative, neutral, and irrelavent labels again and again"""

class Find_freq:
    def __init__ (self,entity):
        self.entity = entity
        
    def calculat_freq(self,label):
        
        dic = {}

        for i in self.entity:
            if i not in dic:
                dic[i] = train[train["entity"]==i][train["labels"]==label]["labels"].count()

        return dic


In [14]:
ent = train["entity"].unique()
# creating obj of above class and passing it list of the entities
find_freq = Find_freq(ent)

"""calling the method of the class and passing it different labels/targets and method is returning dictionary
(entities as keys and labels as there frequencies)"""

dic_pos = find_freq.calculat_freq("Positive")
dic_neg = find_freq.calculat_freq("Negative")
dic_neut = find_freq.calculat_freq("Neutral")
dic_irre = find_freq.calculat_freq("Irrelevant")


In [15]:
# ploting the different frequencies of according to entities
fig=make_subplots(rows=2,cols=2,subplot_titles=("Positive twittes about entities", "Negative twittes about entities",
                                              "  Neutral twittes about entities","Irrelavent twittes about entities"))

fig.add_trace(go.Bar(x=list(dic_pos.keys()),y=list(dic_pos.values())), row=1, col=1)
fig.add_trace(go.Bar(x=list(dic_neg.keys()),y=list(dic_neg.values())), row=1, col=2)
fig.add_trace(go.Bar(x=list(dic_neut.keys()),y=list(dic_neut.values())), row=2, col=1)
fig.add_trace(go.Bar(x=list(dic_irre.keys()),y=list(dic_irre.values())), row=2, col=2)

fig.update_layout(height=1050, width=1050,
                  title={
        'text': "Twittes about entities",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},showlegend=False)

fig.show()

# Data Preprocessing 

In [16]:
train

,0,entity,labels,tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [17]:
y = train["labels"]
X = train.drop(columns=[0,"labels"])

In [18]:
print(len(y))
print(len(X))

# Encoding labels names using label encoder from sklearn.preprocessing
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(np.unique(y))



74682
74682
[0 1 2 3]


In [19]:
# reversing the label encoding
print(np.unique(encoder.inverse_transform(y)))

['Irrelevant' 'Negative' 'Neutral' 'Positive']


In [20]:
nltk.download('wordnet')
nltk.download("stopwords")
stop_words = [elements.lower() for elements in set(stopwords.words("english"))]

[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [21]:
X

,entity,tweets
0,Borderlands,im getting on borderlands and i will murder yo...
1,Borderlands,I am coming to the borders and I will kill you...
2,Borderlands,im getting on borderlands and i will kill you ...
3,Borderlands,im coming on borderlands and i will murder you...
4,Borderlands,im getting on borderlands 2 and i will murder ...
...,...,...
74677,Nvidia,Just realized that the Windows partition of my...
74678,Nvidia,Just realized that my Mac window partition is ...
74679,Nvidia,Just realized the windows partition of my Mac ...
74680,Nvidia,Just realized between the windows partition of...


In [22]:
X.drop(columns="entity", inplace =True)

In [23]:
# Lowering text (tweets) and  replacing any extra/special/digits from tweets using regex
X = X["tweets"].str.lower()
X.replace("[^a-zA-Z]"," ", regex=True, inplace=True)
X.head()

0    im getting on borderlands and i will murder yo...
1    i am coming to the borders and i will kill you...
2    im getting on borderlands and i will kill you ...
3    im coming on borderlands and i will murder you...
4    im getting on borderlands   and i will murder ...
Name: tweets, dtype: object

In [24]:
# Removing stop words from the tweets
def remove_stop (x):
    return ",".join([words for words in str(x).split() if words not in stop_words])

X = X.apply(lambda x: remove_stop(x))
X.head()

0    im,getting,borderlands,murder
1              coming,borders,kill
2      im,getting,borderlands,kill
3     im,coming,borderlands,murder
4    im,getting,borderlands,murder
Name: tweets, dtype: object

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
    test_size=0.2,shuffle = True, random_state = 42)

In [26]:
y_train

array([2, 3, 2, ..., 1, 1, 2])

In [27]:

encoder.inverse_transform(y_train)

array(['Neutral', 'Positive', 'Neutral', ..., 'Negative', 'Negative',
       'Neutral'], dtype=object)

In [28]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(59745,)
(14937,)
(59745,)
(14937,)


In [29]:
X

0                            im,getting,borderlands,murder
1                                      coming,borders,kill
2                              im,getting,borderlands,kill
3                             im,coming,borderlands,murder
4                            im,getting,borderlands,murder
                               ...                        
74677    realized,windows,partition,mac,like,years,behi...
74678    realized,mac,window,partition,years,behind,nvi...
74679    realized,windows,partition,mac,years,behind,nv...
74680    realized,windows,partition,mac,like,years,behi...
74681    like,windows,partition,mac,like,years,behind,d...
Name: tweets, Length: 74682, dtype: object

In [30]:
# creating array/list of the tweets (train and test)
x_train_array = []
for row in range(0, len(X_train.index)):
    x_train_array.append("".join(x for x in X_train.iloc[row] ))
    
    
x_test_array = []
for row in range (0, len(X_test.index)):
    x_test_array.append("".join(x for x in X_test.iloc[row]))

In [31]:
# converting arrays of tweets into vector form
tfidfvector = TfidfVectorizer()
tfidf_train_dataset = tfidfvector.fit_transform(x_train_array)
tfidf_test_dataset = tfidfvector.transform(x_test_array)

In [32]:
encoder.inverse_transform(y_train)

array(['Neutral', 'Positive', 'Neutral', ..., 'Negative', 'Negative',
       'Neutral'], dtype=object)

# Logistic Regression Model

In [33]:
# Model decleration
lr_model = LR(C=10.0, max_iter=100,
                penalty='l2',
                   random_state=10, solver='liblinear' )

# training the model
lr_model.fit(tfidf_train_dataset,y_train)

# testing the model
predictions = lr_model.predict(tfidf_test_dataset)

In [34]:
matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)
kappa = cohen_kappa_score(y_test, predictions)
print(kappa)

[[2073  134  139  246]
 [  60 3991  172  296]
 [  77  200 3034  285]
 [  93  196  191 3750]]
0.8601459463078261
              precision    recall  f1-score   support

           0       0.90      0.80      0.85      2592
           1       0.88      0.88      0.88      4519
           2       0.86      0.84      0.85      3596
           3       0.82      0.89      0.85      4230

    accuracy                           0.86     14937
   macro avg       0.87      0.85      0.86     14937
weighted avg       0.86      0.86      0.86     14937

0.8104727481308267


# Testing the Model

In [35]:
test =  ["Its ok eluno musk given every one a good space to talk","I am ok whatever people will say about anyone","It depends on peoples mind set how they talk either dirty or good mind","pakistans politicians are not doing good job in their country"]
tes = tfidfvector.transform(test)

pre = lr_model.predict(tes)


In [36]:
print(encoder.inverse_transform(pre))

['Positive' 'Negative' 'Irrelevant' 'Neutral']


In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
# forest_model = RandomForestClassifier()
# forest_model.fit(tfidf_train_dataset, y_train)

# prediction = forest_model.predict(tfidf_test_dataset)

# matrix=confusion_matrix(y_test,predictions)
# print(matrix)
# score=accuracy_score(y_test,predictions)
# print(score)
# report=classification_report(y_test,predictions)
# print(report)
# kappa = cohen_kappa_score(y_test, predictions)
# print(kappa)

> Open for suggestions, Looking forward to you guys 😊